<a href="https://colab.research.google.com/github/sosucat/sbl-optimizer-colab/blob/main/sbl_optimizer_jp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 💡 使用するためにはコピーして下さい

このノートは **閲覧のみ**の設定です。
ノートを使うためには、まず**コピーの保存**を行ってください:

- 左上から:  **`ファイル → ドライブにコピーを保存`**

- Googleドライブに作成されたコピーは、編集・実行可能です。

- Googleアカウントにサインインする必要があります。


# 🐐 sbl-optimizer:  [Swell by Light](https://sites.gatech.edu/futurefeelings/2025/07/03/swell-by-light-tei-25-2/)の模様最適化ソフト
[![PyPI version](https://badge.fury.io/py/sbl-optimizer.svg)](https://badge.fury.io/py/sbl-optimizer)
[![sbl-optimizer](https://img.shields.io/badge/sbl--optimizer-black?logo=github)](https://github.com/sosucat/sbl-optimizer)

[![Homepage](https://img.shields.io/badge/🔗_ホームベージ-black)](https://sites.gatech.edu/futurefeelings/2025/07/03/swell-by-light-tei-25-2/)
[![Author](https://img.shields.io/badge/著者サイト-black?logo=googlescholar&logoColor=white)](https://sosuke-ichihashi.com/)
[![Research paper](https://img.shields.io/badge/研究論文-black?logo=acm)](https://doi.org/10.1145/3689050.3704420)
[![Fabrication](https://img.shields.io/badge/🔗_作り方-black)](https://sites.gatech.edu/futurefeelings/2025/07/23/%e5%85%89%e3%81%a7%e3%83%87%e3%82%b3%e3%83%9c%e3%82%b3%e3%82%82%e3%82%88%e3%81%86%e3%82%92%e4%bd%9c%e3%82%8d%e3%81%86%ef%bc%81/)
[![Watch fabrication demo on YouTube](https://img.shields.io/badge/作り方-750014?logo=youtube)](https://youtu.be/LomVS_jHxl0?feature=shared)

**sbl-optimizer** は、Swell by Lightで画像に忠実なデコボコ模様を作るため、画像の濃淡を修正するソフトです。

**本ノートは上級者向け**なので、sbl-optimizerライブラリをPython環境で利用して模様画像を最適化する方法の詳細を理解できます。

**sbl-optimizerをパッと使いたいだけの方は、以下の簡易版をおすすめ**します。
簡易版では、プログラミングの知識がなくても、画像をアップロードしてセルを実行するだけで、最適化された画像を作ることができます。できあがった画像を印刷し、光を当てると、元画像の模様と同じようなデコボコ模様ができあがります。

[![Simpler Colab](https://img.shields.io/badge/簡易版Colabノート-black?logo=googlecolab)](https://colab.research.google.com/drive/15zYmaNvh88jztUcqpzwLXtT4YMRk2i1G?usp=sharing)

---


![A printed pattern's shades change as the optimization progresses, and the resulting temperature distribution gets closer to the intended pattern.](https://sites.gatech.edu/futurefeelings/files/2025/03/opt_step.gif)

画像の模様の濃淡を最適化することで、画像の模様（左）に近い、均一な温度分布（右）になり、画像に近いきれいなデコボコ模様ができます。
この例では、最適化前（iteration: 0）は葉の部分が熱くなりすぎている一方で、茎の部分は加熱不足の状態でした。
最適化が進むにつれて、葉の温度は下がり、茎の温度は上昇していき、結果としてより均一な温度分布が実現されています。

---

## 📑 目次
1. [sbl-optimizerのインストール](#scrollTo=vkd63QF86-g3)  
2. [ライブラリのインポート](#scrollTo=mHjEnEtZ7azV)  
3. [クイックスタート：サンプル画像を最適化](#scrollTo=jEwPKC3COIbX)  
4. [自前の模様を最適化](#scrollTo=ay2GXkzFPJuT)  
5. [設定の変更](#scrollTo=uHP6gd2SYqIg)  
6. [よくある質問 ＆ 問題の解決方法](#scrollTo=-pFXIm8gfsJX)
7. [ライセンス ＆ 引用方法](#scrollTo=q2DhdMj88Kec)
8. [発展](#scrollTo=af-mVoGqfRIf)

---

## 1. sbl-optimizerのインストール
> pipでPyPIから最新のsbl-optimizerをインストールします。

In [ ]:
# Install the latest version from PyPI
!pip install sbl-optimizer

## 2. ライブラリのインポート

> 必要なライブラリをインポートしておきます。

In [ ]:
import json
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image
import importlib.resources as pkg_resources
from google.colab import files
from IPython.display import IFrame, display, Javascript
import base64

from sbl_optimizer.config import Config
from sbl_optimizer.heat_solver import image_to_heat_pattern, optimize
from sbl_optimizer.utils import compute_dims
from sbl_optimizer.io import save_pattern, save_plots

##3. クイックスタート：サンプル画像の最適化
**概要**： 模様画像を最適化し、その結果をダウンロードする方法をお見せします。今回は、sbl-optimizerが提供するサンプルの模様画像を最適化してみます。
1. ⚙️ デフォルト設定の読み込み
2. 🖼️ サンプル画像の読み込み
3. 🔧 模様画像の事前処理
4. 👀 事前処理されたサンプル画像の確認
5. 🔁 最適化の実行
6. 👀 最適化された模様画像の確認
7. 👀 予想されるデコボコ模様と温度分布の確認
8. 💾 最適化された模様画像のダウンロード
9. 📉 （任意）最適化パフォーマンスの確認

### 3-1. ⚙️ デフォルト設定の読み込み


> まず、最適化で使用するパラメータ（ペーストが膨張する温度やシミュレーションの解像度など）を決めます。今回はデフォルトの設定にしておきます。



In [ ]:
# Locate the bundled config.json inside the installed package
def get_default_config_path():
    return pkg_resources.as_file(
        pkg_resources.files("sbl_optimizer.assets") / "config.json"
    )

with get_default_config_path() as config_path:
    with open(config_path) as f:
        cfg_data = json.load(f)

print("Default configuration:")
for k, v in cfg_data.items():
    print(f"- {k}: {v}")

# Create Config object
cfg = Config.from_file(config_path)

### 3-2. 🖼️ サンプル画像の読み込み
> 今回は最適化実行の一連の手順を見ることを優先し、ライブラリ付属のサンプル画像を読み込みます。

In [ ]:
with pkg_resources.as_file(
    pkg_resources.files("sbl_optimizer.assets") / "sample.jpg"
) as sample_img_path:
    img = Image.open(sample_img_path)
    plt.figure(figsize=(4,4))
    plt.imshow(img)
    plt.axis('off')
    plt.title("Sample Input Image")
    plt.show()

### 3-3. 🔧 模様画像の事前処理


> 模様画像の模様部分を均一な灰色にしておきます。また、シミュレーションの解像度に合わせて模様画像の解像度を変更します。


In [ ]:
mask, H_init, max_heat = image_to_heat_pattern(sample_img_path, cfg)

### 3-4. 👀 事前処理されたサンプル画像の確認
> 事前処理されたサンプル画像を表示します。これが最適化前の状態です。



In [ ]:
# Visualize the initialized print pattern using matplotlib
def visualize_init_pattern(H_init, max_heat):
  plt.figure(figsize=(4,4))
  plt.imshow(H_init/max_heat, cmap='gray_r', vmin=0.0, vmax=1.0)
  plt.axis('off')
  plt.title("Initial print pattern")
  plt.show()

visualize_init_pattern(H_init, max_heat)

### 3-5. 🔁 最適化の実行

> この最適化の目的は、紙内で熱伝導が起こっても元の模様に近いデコボコ模様をつくることです。模様をそのまま均一なグレーや黒にすると、熱伝導の影響で太い線は熱くなりすぎ、細い線は熱くならないため、画像通りの温度分布にならず、温度分布に基づくデコボコ模様も画像通りになりません。なので、熱伝導の結果が元画像通りの温度分布になるように「最適化」を行い、部分ごとの濃淡を調節します。


In [ ]:
%%time
# Set the maximum height of the output
display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 200})'))

# Compute physical dimensions (for logging)
img_w, img_h, phys_w, phys_h = compute_dims(Image.open(sample_img_path))

# Optimize
T_best, H_best, errors = optimize(mask, H_init, max_heat, img_w, img_h, phys_w, phys_h, cfg)
print(f"Optimization completed in {len(errors)} iterations.")

###3-6. 👀 最適化された模様画像の確認

> 最適化された模様画像を保存・表示します。ここで最適化された画像に満足であれば、後でダウンロードします。



In [ ]:
# Save pattern PDF
pattern_pdf = save_pattern(sample_img_path, H_best, max_heat)
print(f"Saved optimized pattern to: {pattern_pdf}")

# Display the generated PDF inline in Colab
with open(pattern_pdf, "rb") as pdf_file:
    encoded_pdf = base64.b64encode(pdf_file.read()).decode('utf-8')
IFrame(f"data:application/pdf;base64,{encoded_pdf}", width=400, height=400)

### 3-7. 👀 予想されるデコボコ模様と温度分布の確認

> 予想されるデコボコ模様と温度分布を確認できます。



In [ ]:
def display_plots(plots):
  fig, (ax_swell, ax_temperature) = plt.subplots(1, 2, figsize=(4 * 2, 4))

  img_plot = Image.open(plots[0])
  ax_swell.imshow(img_plot)
  ax_swell.axis('off')
  ax_swell.set_title('Swell Pattern')

  img_plot = Image.open(plots[1])
  ax_temperature.imshow(img_plot)
  ax_temperature.axis('off')
  ax_temperature.set_title('Temperature Distribution [C°]')

  plt.tight_layout()
  plt.show()

# Save and display plots with matplotlib
plots = save_plots(T_best, phys_w, phys_h, sample_img_path, dpi=72)
display_plots(plots)

### 3-8. 💾 最適化された模様画像のダウンロード

> 結果に満足であれば、最適化された模様画像をダウンロードします。

In [ ]:
# Ask user if they want to download the PDF locally
def download_pdf(pdf_file):
  download_choice = input('Do you want to download the optimized PDF? (y/n): ').strip().lower()
  if download_choice == 'y':
      files.download(str(pdf_file))

download_pdf(pattern_pdf)

###3-9. 📉（任意）最適化パフォーマンスの確認

> 興味があれば、最適化において模様がどのように改善されたかを確認できます。「Outlier Cells」とは、光加熱後に元の模様通りにならなかった部分の割合です。つまり、これが0になれば、元画像の模様とデコボコ模様が（シミュレーション上では）完全に一致します。

In [ ]:
# Plot normalized error progression over iterations
def plot_error_progression(errors, cfg):
  errors_norm = [e * 100 / cfg.resolution for e in errors]
  plt.figure(figsize=(6,4))
  plt.plot(errors_norm)
  plt.xlabel('Iteration')
  plt.ylabel('Outlier Cells [%]')
  plt.title('Optimization Error Progress')
  plt.show()

plot_error_progression(errors, cfg)

## 4. 自前の模様を最適化
**概要**：自前の模様画像を最適化しましょう。
0. **3章をスキップした場合**： 3章の関数を定義
1. 📁 模様画像のアップロード
2. 🔧 模様画像の事前処理と確認
3. 🔁 最適化の実行
4. 👀 最適化された模様画像の確認
5. 👀 予想されるデコボコ模様と温度分布の確認
6. 💾 最適化された模様画像のダウンロード
7. 📉 （任意）最適化パフォーマンスの確認

### 4-0. **3章をスキップした場合**： 3章の関数を定義
> 3章を実行せずにいきなりこの章に進んだ場合でも、このセルを実行すれば問題なく以後のセルを実行できます。

In [ ]:
# Locate the bundled config.json inside the installed package
def get_default_config_path():
    return pkg_resources.as_file(
        pkg_resources.files("sbl_optimizer.assets") / "config.json"
    )

# Visualize the initialized print pattern using matplotlib
def visualize_init_pattern(H_init, max_heat):
  plt.figure(figsize=(4,4))
  plt.imshow(H_init/max_heat, cmap='gray_r', vmin=0.0, vmax=1.0)
  plt.axis('off')
  plt.title("Initial print pattern")
  plt.show()

# Visualize the swell pattern and temperature distribution using matplotlib heatmap
def display_plots(plots):
  fig, (ax_swell, ax_temperature) = plt.subplots(1, 2, figsize=(4 * 2, 4))
  img_plot = Image.open(plots[0])
  ax_swell.imshow(img_plot)
  ax_swell.axis('off')
  ax_swell.set_title('Swell Pattern')
  img_plot = Image.open(plots[1])
  ax_temperature.imshow(img_plot)
  ax_temperature.axis('off')
  ax_temperature.set_title('Temperature Distribution [C°]')
  plt.tight_layout()
  plt.show()

# Ask user if they want to download the PDF locally
def download_pdf(pdf_file):
  download_choice = input('Do you want to download the optimized PDF? (y/n): ').strip().lower()
  if download_choice == 'y':
      files.download(str(pdf_file))

# Plot normalized error progression over iterations
def plot_error_progression(errors, cfg):
  errors_norm = [e * 100 / cfg.resolution for e in errors]
  plt.figure(figsize=(6,4))
  plt.plot(errors_norm)
  plt.xlabel('Iteration')
  plt.ylabel('Outlier Cells [%]')
  plt.title('Optimization Error Progress')
  plt.show()


# Load the default configuration file
with get_default_config_path() as config_path:
    with open(config_path) as f:
        cfg_data = json.load(f)
print("Default configuration:")
for k, v in cfg_data.items():
    print(f"- {k}: {v}")
# Create Config object
cfg = Config.from_file(config_path)

### 4-1. 📁 模様画像のアップロード
> パソコン上からJPGかPNGフォーマットの模様画像をアップロードしてください。 「ファイルを選択」 をクリックして画像を選ぶと、その画像が下に表示されます。

In [ ]:
def upload_image():
  """
  Prompt the user to upload an image and save it to the working directory.
  Returns:
      Path object pointing to the uploaded image file.
  """
  uploaded = files.upload()  # Opens upload dialog
  if not uploaded:
      raise ValueError("No file uploaded.")
  # Get the first uploaded filename
  filename = next(iter(uploaded.keys()))
  print(f"Uploaded file: {filename}")
  return Path(filename)

your_img_path = upload_image()
your_img = Image.open(your_img_path)
plt.figure(figsize=(4,4))
plt.imshow(your_img)
plt.axis('off')
plt.title("Uploaded Input Image")
plt.show()

### 4-2. 👀 事前処理された画像の確認
> 事前処理された画像を表示します。これが最適化前の状態です。



In [ ]:
mask_yours, H_init_yours, max_heat_yours = image_to_heat_pattern(your_img_path, cfg)
visualize_init_pattern(H_init_yours, max_heat_yours)

### 4-3. 🔁 最適化の実行

> この最適化の目的は、紙内で熱伝導が起こっても元の模様に近いデコボコ模様をつくることです。模様をそのまま均一なグレーや黒にすると、熱伝導の影響で太い線は熱くなりすぎ、細い線は熱くならないため、画像通りの温度分布にならず、温度分布に基づくデコボコ模様も画像通りになりません。なので、熱伝導の結果が元画像通りの温度分布になるように「最適化」を行い、部分ごとの濃淡を調節します。


In [ ]:
%%time
# Set the maximum height of the output
display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 200})'))

# Compute physical dimensions (for logging)
img_w_yours, img_h_yours, phys_w_yours, phys_h_yours = compute_dims(Image.open(your_img_path))

# Optimize
T_best_yours, H_best_yours, errors_yours = optimize(mask_yours, H_init_yours, max_heat_yours, img_w_yours, img_h_yours, phys_w_yours, phys_h_yours, cfg)
print(f"Optimization completed in {len(errors_yours)} iterations.")

### 4-4. 👀 最適化された模様画像の確認

> 最適化された模様画像を保存・表示します。ここで最適化された画像に満足であれば、後でダウンロードします。

In [ ]:
# Save pattern PDF
pattern_pdf_yours = save_pattern(your_img_path, H_best_yours, max_heat_yours)
print(f"Saved optimized pattern to: {pattern_pdf_yours}")

# Display the generated PDF inline in Colab
with open(pattern_pdf_yours, "rb") as pdf_file:
    encoded_pdf_yours = base64.b64encode(pdf_file.read()).decode('utf-8')
IFrame(f"data:application/pdf;base64,{encoded_pdf_yours}", width=400, height=400)

###4-5. 👀 予想されるデコボコ模様と温度分布の確認

> 予想されるデコボコ模様と温度分布を確認できます。

In [ ]:
# Save and display plots with matplotlib
plots_yours = save_plots(T_best_yours, phys_w_yours, phys_h_yours, your_img_path, dpi=72)
display_plots(plots_yours)

###4-6. 💾 最適化された模様画像のダウンロード

> 結果に満足であれば、最適化された模様画像をダウンロードします。

In [ ]:
download_pdf(pattern_pdf_yours)

###4-7. 📉（任意）最適化パフォーマンスの確認

> 興味があれば、最適化において模様がどのように改善されたかを確認できます。「Outlier Cells」とは、光加熱後に元の模様通りにならなかった部分の割合です。つまり、これが0になれば、元画像の模様とデコボコ模様が（シミュレーション上では）完全に一致します。

In [ ]:
plot_error_progression(errors_yours, cfg)

##5. 設定の変更
**概要:** シミュレーションの設定をカスタマイズして、自前の模様画像を最適化します。
0. **3章と4章をスキップした場合**：関数の定義
1. ⚙️ 設定の変更
2. 📁 模様画像のアップロード
3. 🔧 模様画像の事前処理と確認
4. 🔁 カスタマイズ設定で最適化の実行
5. 👀 最適化された模様画像の確認
6. 👀 予想されるデコボコ模様と温度分布の確認
7. 💾 最適化された模様画像のダウンロード
8. 📉（任意）最適化パフォーマンスの確認

###5-0. **3章と4章をスキップした場合**：関数の定義
>3章・4章を実行せずにいきなりこの章に進んだ場合でも、このセルを実行すれば問題なく以後のセルを実行できます。

In [ ]:
# Locate the bundled config.json inside the installed package
def get_default_config_path():
    return pkg_resources.as_file(
        pkg_resources.files("sbl_optimizer.assets") / "config.json"
    )

# Visualize the initialized print pattern using matplotlib
def visualize_init_pattern(H_init, max_heat):
  plt.figure(figsize=(4,4))
  plt.imshow(H_init/max_heat, cmap='gray_r', vmin=0.0, vmax=1.0)
  plt.axis('off')
  plt.title("Initial print pattern")
  plt.show()

# Visualize the swell pattern and temperature distribution using matplotlib heatmap
def display_plots(plots):
  fig, (ax_swell, ax_temperature) = plt.subplots(1, 2, figsize=(4 * 2, 4))
  img_plot = Image.open(plots[0])
  ax_swell.imshow(img_plot)
  ax_swell.axis('off')
  ax_swell.set_title('Swell Pattern')
  img_plot = Image.open(plots[1])
  ax_temperature.imshow(img_plot)
  ax_temperature.axis('off')
  ax_temperature.set_title('Temperature Distribution [C°]')
  plt.tight_layout()
  plt.show()

# Ask user if they want to download the PDF locally
def download_pdf(pdf_file):
  download_choice = input('Do you want to download the optimized PDF? (y/n): ').strip().lower()
  if download_choice == 'y':
      files.download(str(pdf_file))

# Plot normalized error progression over iterations
def plot_error_progression(errors, cfg):
  errors_norm = [e * 100 / cfg.resolution for e in errors]
  plt.figure(figsize=(6,4))
  plt.plot(errors_norm)
  plt.xlabel('Iteration')
  plt.ylabel('Outlier Cells [%]')
  plt.title('Optimization Error Progress')
  plt.show()

# Upload image
def upload_image():
  """
  Prompt the user to upload an image and save it to the working directory.
  Returns:
      Path object pointing to the uploaded image file.
  """
  uploaded = files.upload()  # Opens upload dialog
  if not uploaded:
      raise ValueError("No file uploaded.")
  # Get the first uploaded filename
  filename = next(iter(uploaded.keys()))
  print(f"Uploaded file: {filename}")
  return Path(filename)

###5-1. ⚙️ 設定の変更

| パラメータ | 種類 | デフォルト | 説明 |
|-----|------|---------|-------------|
| `swell_temperature` | float | 145 [C°] | 膨張温度。使用する膨張ペーストに記載されている、ペーストが膨らみ始める温度に応じて調整してください。 |
| `light_power`       | float | 100.0 [W] |光出力。使用するライトの出力に応じて調節してください。 |
| `light_diameter`    | float | 0.06 [m] | 紙面に当たる光の直径。紙に光を当てて加熱する際に、光が当たる範囲に応じて調整してください。 |
| `alpha`             | float | 5e-07 [m²/s] | 紙の熱流量。厚紙や極端に薄い紙などを使用する際に調節してください。 |
| `verbose`           | int   | 1 | ログ出力の有無。 0：ログなし。 1: ログあり。 |
| `resolution`        | int   | 120000 | 熱シミュレーションの解像度。解像度を下げれば、最適化速度が上がる。解像度を上げれば、より画像に忠実なデコボコ模様になる。 |

> まずはデフォルト値のまま使ってみて、思うようなデコボコ模様ができなければ、変更するのがよいと思います。使用する材料に応じて、以下のセル内の数値を直接編集してください。



In [ ]:
custom_cfg_contents = {
    "swell_temperature": 140.0,
    "light_power": 120.0,
    "light_diameter": 0.08,
    "alpha": 5e-07,
    "verbose": 0,
    "resolution": 240000
}

In [ ]:
# Write to a JSON file in the notebook workspace
custom_config_path = Path("custom_config.json")
with open(custom_config_path, "w") as f:
    json.dump(custom_cfg_contents, f, indent=4)
print(f"Custom config written to: {custom_config_path}")

# Load and instantiate Config from this custom file
cfg_custom = Config.from_file(custom_config_path)
print("Loaded custom Config:")
for k, v in custom_cfg_contents.items():
    print(f"- {k}: {v}")

###5-2. 📁 模様画像のアップロード
> パソコン上からJPGかPNGフォーマットの模様画像をアップロードしてください。
「ファイルを選択」 をクリックして画像を選ぶと、その画像が下に表示されます。

In [ ]:
your_img_path_c = upload_image()
your_img_c = Image.open(your_img_path_c)
plt.figure(figsize=(4,4))
plt.imshow(your_img_c)
plt.axis('off')
plt.title("Uploaded Input Image")
plt.show()

### 5-3. 🔧 事前処理された画像の確認
> 事前処理された画像を表示します。これが最適化前の状態です。

In [ ]:
mask_yours_c, H_init_yours_c, max_heat_yours_c = image_to_heat_pattern(your_img_path_c, cfg_custom)
visualize_init_pattern(H_init_yours_c, max_heat_yours_c)

###5-4. 🔁 カスタマイズ設定での最適化の実行
熱伝導のせいで期待通りの温度にならないところを減らすため、模様の濃淡を最適化します。シミュレーションは、[ここ](#scrollTo=jNyYGd1sY4Z_)で設定したパラメータに基づいて行われます。ここでの「最適化」とは、光による加熱で浮き出るデコボコ模様が、元の模様と一致するようにすることを意味します。

In [ ]:
%%time
# Set the maximum height of the output
display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 200})'))

# Compute physical dimensions (for logging)
img_w_yours_c, img_h_yours_c, phys_w_yours_c, phys_h_yours_c = compute_dims(Image.open(your_img_path_c))

# Optimize
T_best_yours_c, H_best_yours_c, errors_yours_c = optimize(mask_yours_c, H_init_yours_c, max_heat_yours_c, img_w_yours_c, img_h_yours_c, phys_w_yours_c, phys_h_yours_c, cfg_custom)
print(f"Optimization completed in {len(errors_yours_c)} iterations.")

### 5-5. 👀 最適化された模様画像の確認

> 最適化された模様画像を保存・表示します。ここで最適化された画像に満足であれば、後でダウンロードします。

In [ ]:
# Save pattern PDF
pattern_pdf_yours_c = save_pattern(your_img_path_c, H_best_yours_c, max_heat_yours_c)
print(f"Saved optimized pattern to: {pattern_pdf_yours_c}")

# Display the generated PDF inline in Colab
with open(pattern_pdf_yours_c, "rb") as pdf_file:
    encoded_pdf_yours_c = base64.b64encode(pdf_file.read()).decode('utf-8')
IFrame(f"data:application/pdf;base64,{encoded_pdf_yours_c}", width=400, height=400)

###5-6. 👀 予想されるデコボコ模様と温度分布の確認

> 予想されるデコボコ模様と温度分布を確認できます。

In [ ]:
# Save and display plots with matplotlib
plots_yours_c = save_plots(T_best_yours_c, phys_w_yours_c, phys_h_yours_c, your_img_path_c, dpi=72)
display_plots(plots_yours_c)

###5-7. 💾 最適化された模様画像のダウンロード

> 結果に満足であれば、最適化された模様画像をダウンロードします。

In [ ]:
download_pdf(pattern_pdf_yours_c)

###5-8. 📉（任意）最適化パフォーマンスの確認

> 興味があれば、最適化において模様がどのように改善されたかを確認できます。「Outlier Cells」とは、光加熱後に元の模様通りにならなかった部分の割合です。つまり、これが0になれば、元画像の模様とデコボコ模様が（シミュレーション上では）完全に一致します。

In [ ]:
plot_error_progression(errors_yours_c, cfg_custom)

## 6. よくある質問 ＆ 問題の解決方法

Q: どのファイル形式に対応していますか？
A: JPG または PNG 画像をアップロードできます。

Q: 画像がアップロードできません。
A: ファイルを選択 ボタンをクリックして、有効な画像ファイルを選んでください。うまくいかない場合はセルを再実行してください。

Q: 最適化中にエラーが出ました。
A: ファイルが大きすぎないか確認してください。より小さな画像を使い、再実行してみてください。

Q: 最適化にはどのくらい時間がかかりますか？
A: Colab 上では、1～2分ほどかかる場合があります。

ヒント： うまく動作しない場合は、ノートブックを再起動（「ランタイム」→「ランタイムを再起動」）し、最初から再実行してみてください。

以上で問題が解決しない場合、以下からお問い合わせください。

[![@RefreshSource](https://img.shields.io/badge/@RefreshSource-black?logo=x&logoColor=white)](https://x.com/refreshsource)
[![Email](https://img.shields.io/badge/sosuke.ichihashi@gmail.com-black?logo=gmail&logoColor=white)](mailto:sosuke.ichihashi@gmail.com)

## 7. ライセンス ＆ 引用方法
###ライセンス
>本プロジェクトは、[MITライセンス](https://github.com/sosucat/sbl-optimizer-colab/blob/main/LICENSE)のもとで公開されています.


###開発者
>市橋 爽介

> [![市橋 爽介](https://img.shields.io/badge/市橋のサイト-black?logo=googlescholar&logoColor=white)](https://sosuke-ichihashi.com/)
[![@sosucat](https://img.shields.io/badge/@sosucat-black?logo=github&logoColor=white)](https://github.com/sosucat)
[![@RefreshSource](https://img.shields.io/badge/@RefreshSource-black?logo=x&logoColor=white)](https://x.com/refreshsource)


###引用方法
> 研究やプロジェクトで **sbl-optimizer** を使用される場合は、以下を引用してください:
* Sosuke Ichihashi, Noura Howell, and HyunJoo Oh. 2025.\
Swell by Light: An Approachable Technique for Freeform Raised Textures. \
In Proceedings of the Nineteenth International Conference on Tangible, Embedded, and Embodied Interaction (TEI '25). Association for Computing Machinery, New York, NY, USA, Article 45, 1–16. https://doi.org/10.1145/3689050.3704420
```bibtex
@inproceedings{10.1145/3689050.3704420,
author = {Ichihashi, Sosuke and Howell, Noura and Oh, HyunJoo},
title = {Swell by Light: An Approachable Technique for Freeform Raised Textures},
year = {2025},
isbn = {9798400711978},
publisher = {Association for Computing Machinery},
address = {New York, NY, USA},
url = {https://doi.org/10.1145/3689050.3704420},
doi = {10.1145/3689050.3704420},
booktitle = {Proceedings of the Nineteenth International Conference on Tangible, Embedded, and Embodied Interaction},
articleno = {45},
numpages = {16},
keywords = {2.5D fabrication, Personal fabrication, tactile rendering},
location = {Bordeaux / Talence, France},
series = {TEI '25}
}
```

## 8. **もっと深く知りたい方は、以下をどうぞ！**


🎨 **Swell by Light によるデコボコ模様の作り方**

> ウェブサイトと動画で、デコボコ模様の作り方を紹介しています。

>[![Fabrication](https://img.shields.io/badge/🔗_作り方-black)](https://sites.gatech.edu/futurefeelings/2025/07/23/%e5%85%89%e3%81%a7%e3%83%87%e3%82%b3%e3%83%9c%e3%82%b3%e3%82%82%e3%82%88%e3%81%86%e3%82%92%e4%bd%9c%e3%82%8d%e3%81%86%ef%bc%81/)
[![Watch fabrication demo on YouTube](https://img.shields.io/badge/作り方-750014?logo=youtube)](https://youtu.be/LomVS_jHxl0?feature=shared)


🔗 **ホームページ**

> Swell by Lightのホームページで、研究詳細や応用例を紹介しています。

>[![Project page](https://img.shields.io/badge/🔗_ホームページ-black)](https://sites.gatech.edu/futurefeelings/2025/07/03/swell-by-light-tei-25-2/)


📄 **研究論文**

> Swell by Lightの理論や関連研究、応用例については、論文をご参照ください。

>[![Research paper](https://img.shields.io/badge/研究論文-black?logo=acm)](https://doi.org/10.1145/3689050.3704420)


👩‍💻 **ローカル実行用のコマンドラインツール**

> PyPI や GitHub にあるチュートリアルを見て、お使いのパソコン上で sbl-optimizer をより簡単に使ってみましょう。

>[![PyPI version](https://badge.fury.io/py/sbl-optimizer.svg)](https://badge.fury.io/py/sbl-optimizer)
[![GitHub](https://img.shields.io/badge/sbl--optimizer-black?logo=github)](https://github.com/sosucat/sbl-optimizer)


🦙 **簡易版Colabノート**

> 本ノートで紹介した機能を手軽に使用したい方は、こちらをご覧ください。

>[![Simple Colab](https://img.shields.io/badge/簡易版Colabノート-black?logo=googlecolab)](https://colab.research.google.com/drive/15zYmaNvh88jztUcqpzwLXtT4YMRk2i1G?usp=sharing)


👨‍💻 **ソースコードと問題報告**

> 問題の報告や貢献、最新の開発状況の確認には、ぜひ GitHub リポをご覧ください。

>[![GitHub sbl-optimizer](https://img.shields.io/badge/sbl--optimizer-black?logo=github)](https://github.com/sosucat/sbl-optimizer)
[![GitHub sbl-optimizer-colab](https://img.shields.io/badge/このColabノート-black?logo=github)](https://github.com/sosucat/sbl-optimizer-colab)


🕵️‍♂️ **著者のサイト**

> 市橋 爽介の他の研究もぜひご覧あれ！

>[![著者のサイト](https://img.shields.io/badge/市橋のサイト-black?logo=googlescholar&logoColor=white)](https://sosuke-ichihashi.com/)


*よいモノづくりを！*